<a href="https://colab.research.google.com/github/pratyush981/Portfolio/blob/main/CUDA%20Kernels%20with%20NUMBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):

    idx = cuda.grid(1)
    out[idx] = x[idx] + y[idx]

In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)
threads_per_block = 128
blocks_per_grid = 32

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host())

[   1    2    3 ... 4094 4095 4096]


In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)
threads_per_block = 32
blocks_per_grid = 32

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host())
print(d_out.copy_to_host()[1023], d_out.copy_to_host()[1024])

In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)
threads_per_block = 128
blocks_per_grid = 16

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host())
print(d_out.copy_to_host()[2047], d_out.copy_to_host()[2048])

In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)
threads_per_block = 256
blocks_per_grid = 64

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
cuda.synchronize()
print(d_out.copy_to_host())



[   1    2    3 ... 4094 4095 4096]


In [ ]:
import numpy as np

n = 4096
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)
threads_per_block = 128
blocks_per_grid = 32

In [ ]:

"""
def square_device(a):
    return a**2
"""

'\ndef square_device(a):\n    return a**2\n'

In [ ]:

@cuda.jit
def square_device(a, out):
    idx = cuda.grid(1)
    out[idx] = a[idx] ** 2

In [ ]:

n = 4096

a = np.arange(n)
out = a**2

In [ ]:
d_a = cuda.to_device(a)
d_out = cuda.device_array_like(d_a)
blocks = 32
threads = 128



square_device[blocks, threads](d_a, d_out)

In [ ]:
from numpy import testing
testing.assert_almost_equal(d_out, out)

In [ ]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):


    start = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [ ]:
import numpy as np

n = 100000
x = np.arange(n).astype(np.int32)
y = np.ones_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_out = cuda.device_array_like(d_x)

threads_per_block = 128
blocks_per_grid = 30

In [ ]:
add_kernel[blocks_per_grid, threads_per_block](d_x, d_y, d_out)
print(d_out.copy_to_host())

[     1      2      3 ...  99998  99999 100000]


In [ ]:
"""
from math import hypot

def hypot_stride(a, b, c):
    c = hypot(a, b)
"""

'\nfrom math import hypot\n\ndef hypot_stride(a, b, c):\n    c = hypot(a, b)\n'

In [ ]:
from math import hypot

@cuda.jit
def hypot_stride(a, b, c):
    idx = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(idx, a.shape[0], stride):
        c[i] = hypot(a[i], b[i])


In [ ]:

n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)

In [ ]:
from numpy import testing
testing.assert_almost_equal(np.hypot(a,b), d_c.copy_to_host(), decimal=5)

In [ ]:
from numba import jit

@jit
def numba_hypot(a, b):
    return np.hypot(a, b)

In [ ]:
%timeit numba_hypot(a, b)

4.97 ms ± 6.75 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%time hypot_stride[128, 64](d_a, d_b, d_c); cuda.synchronize()

In [ ]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # This is bad

@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)

In [ ]:
# This gets the wrong answer
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [1]


In [ ]:
# This works correctly
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [4096]


In [ ]:
def cpu_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    for element in x:
        bin_number = np.int32((element - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            histogram_out[bin_number] += 1

In [ ]:
x = np.random.normal(size=10000, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

cpu_histogram(x, xmin, xmax, histogram_out)

histogram_out

In [ ]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins

    for i in range(start, x.shape[0], stride):
        bin_number = np.int32((x[i] - xmin) / bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            cuda.atomic.add(histogram_out, bin_number, 1)

In [ ]:
d_x = cuda.to_device(x)
d_histogram_out = cuda.to_device(np.zeros(shape=10, dtype=np.int32))

blocks = 128
threads_per_block = 64

cuda_histogram[blocks, threads_per_block](d_x, xmin, xmax, d_histogram_out)

In [ ]:

np.testing.assert_array_almost_equal(d_histogram_out.copy_to_host(), histogram_out, decimal=2)

In [ ]:
import numpy as np
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)

In [ ]:
@cuda.jit
def monte_carlo_mean(rng_states, iterations, out):
    thread_id = cuda.grid(1)
    total = 0
    for i in range(iterations):
        sample = xoroshiro128p_uniform_float32(rng_states, thread_id)
        total += sample

    out[thread_id] = total/iterations

In [ ]:
out = cuda.device_array(threads_per_block * blocks, dtype=np.float32)
monte_carlo_mean[blocks, threads_per_block](rng_states, 10000, out)
print(out.copy_to_host().mean())

0.50000983


In [ ]:
from numba import njit
import random

@njit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
nsamples = 10000000
%timeit monte_carlo_pi(nsamples)

105 ms ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from numba import njit
import random

@njit
def monte_carlo_pi_device(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:

nsamples = 10000000
threads_per_block = 128
blocks = 32

grid_size = threads_per_block * blocks
samples_per_thread = int(nsamples / grid_size)

rng_states = create_xoroshiro128p_states(grid_size, seed=1)
d_out = cuda.device_array(threads_per_block * blocks, dtype=np.float32)

In [ ]:
print(d_out.copy_to_host().mean())